TOPIX500銘柄の取得

In [ ]:
from tqdm import tqdm
import pandas as pd
import jquantsapi
import os
import warnings

warnings.simplefilter("ignore")

import matplotlib.pyplot as plt

data_list = []
for path in sorted(os.listdir("../data/jquants_api_20221128/")):
    data_list.append(pd.read_parquet("../data/jquants_api_20221128/" + path))

topix_500 = pd.concat(data_list)

df = topix_500.copy()
df = df.sort_values(["Code", "Date"]).reset_index(drop=True)
df["close_change"] = df.groupby(["Code"])["AdjustmentClose"].pct_change()
df = df.set_index(["Code", "Date"])[["close_change"]].unstack(-1)

# 欠損を持つ対象は除外する
df = df.iloc[:, 1:].dropna()

クラスタリングの実装

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
Z = linkage(df, method='ward') 

系列情報を表示

In [ ]:
plt.figure(figsize=(16, 6))
dendrogram(Z, labels=list(df.index))
plt.title("Dedrogram")
plt.show()

最後の方だけ表示

In [ ]:
plt.figure(figsize=(16, 6))
dendrogram(Z, labels=list(df.index), truncate_mode="lastp", p=30)
plt.title("Dedrogram")
plt.show()

クラスタ番号の取得

In [ ]:
clusters = fcluster(Z, 30, criterion="maxclust")
cluster_dict = dict(zip(df.index.values, clusters))

6532の株価推移の表示

In [ ]:
topix_500["cluster"] = topix_500["Code"].map(cluster_dict)
topix_500[topix_500["Code"] == "65320"].plot(
    x="Date", y="AdjustmentClose", figsize=(12, 4), title="65320"
)

JQuantsAPIから銘柄情報を取得

In [ ]:
my_mail_address:str = "***"
my_password: str = "***"

cli = jquantsapi.Client(mail_address=my_mail_address, password=my_password)

info_df = cli.get_listed_info()
info_df["cluster"] = info_df["Code"].map(cluster_dict)
info_df = info_df[~info_df["cluster"].isnull()].reset_index(drop=True)